# Places for Half Earth Future
#### This is a new feature that will show precalculated biodiversity and contextual data for selected geometries around the globe (Up to 20 places per country)
#### Following this first approach there might be more narrative involved and a diffeent approach to how to show the data. Right now, the layer will be accessed from the 'Explore places' tab.

In [5]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy

In [6]:
path = '../../HE_Data/PlacesHE'

In [ ]:
#Columba livia, SliceNumber = 2233
#gadm1= pd.read_csv('/Users/tamarahuete/Documents/HALF_EARTH/gadm1_precalc_no_geometry.csv')
#json.loads(gadm1[gadm1['NAME_1']=='Nyeri']['birds'].values[0])
#{'SliceNumber': 2330.0, 'percentage_presence': 85.8}

In [7]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [8]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [9]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


### 20 places geometries

In [10]:
places= gpd.read_file('{0}/Geometries/Top_20_places_per_country_20211123.shp'.format(path)) # Scott's geometry
places.head()

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry
0,AFG,Afghanistan,1,60.222222,7687.504920,"POLYGON ((6784878.655 4081461.837, 6783664.553..."
1,AFG,Afghanistan,2,60.222222,4706.088317,"POLYGON ((6833826.327 4162440.126, 6832851.109..."
2,AFG,Afghanistan,3,52.312500,11768.366965,"POLYGON ((6641029.409 4066345.857, 6640031.675..."
3,AFG,Afghanistan,4,45.500000,205.123344,"POLYGON ((6544864.953 4033762.857, 6544087.481..."
4,AFG,Afghanistan,5,45.500000,205.123344,"POLYGON ((6520743.383 4065965.621, 6519965.911..."


In [11]:
places['MOL_ID']= places.index+1

#### Format Biodiversity data

In [12]:
### BIOdiversity 

dff = places
dff['reptiles']=np.nan
dff['amphibians']=np.nan
dff['mammals']=np.nan
dff['birds']=np.nan


### Paths to data generate inside ArcGIS Pro
paths = {'amphibians':'{0}/Csvs/Sample_amph_nozeroes.csv'.format(path),
         'birds':'{0}/Csvs/Sample_birds_nozeroes.csv'.format(path),
         'mammals':'{0}/Csvs/Sample_mammals_nozeroes.csv'.format(path),
         'reptiles':'{0}/Csvs/Sample_reptiles_nozeroes.csv'.format(path)}

### Ids of lookup tables for each taxa in ArcGIS online
lookups = {'amphibians':'a641a4cd269345dea93b8bcb1cb66676',
         'birds':'4d8698734b654bb9bb7a61d9af314c76',
         'mammals':'84d3c71caf97479d85f620a4ee217d68',
         'reptiles':'b720e84869624e5482b25338b27a54f2'}

In [13]:
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

In [11]:
%%time
### Loop over each set of data and add all results to one dictionary that will be a text field in the Feature Service
for taxa in paths.keys():
    print(taxa) 
    df = pd.read_csv(paths[taxa])
        
    ### Get species area against global species range:
    lookup = getHTfromId(lookups[taxa])
    df = df.merge(lookup[['SliceNumber','range_area_km2']], how='left',on = 'SliceNumber')
    df = df.merge(places[['MOL_ID','AREA_KM2']])
    
    if taxa == 'mammals':
        df['per_global'] = round(df['presence']/df['range_area_km2']*100,2)
        df['per_aoi'] = round(df['presence']/df['AREA_KM2']*100,2)
    else:
        df['per_global'] = round(df[taxa]/df['range_area_km2']*100,2)
        df['per_aoi'] = round(df[taxa]/df['AREA_KM2']*100,2)
    
    ### Format as array of dicts:
    for i in dff.MOL_ID:
        species = []
        for j in df[(df.MOL_ID== i)].SliceNumber.unique():
            sp_dict = {'SliceNumber':int(df[(df.MOL_ID== i) & (df.SliceNumber==j)].SliceNumber.values[0]),
                       'per_global':df[(df.MOL_ID== i) & (df.SliceNumber==j)].per_global.values[0],
                       'per_aoi':df[(df.MOL_ID== i) & (df.SliceNumber==j)].per_aoi.values[0]
                      }
            species.append(sp_dict)
        dff.loc[dff['MOL_ID']==i,taxa] =json.dumps(species)
    print(f'finished MOL_ID ={i}')
pd.set_option('display.max_colwidth',None)
test= pd.DataFrame(dff[dff.columns[~dff.columns.isin(['geometry'])]])
dff.to_file("../../HE_Data/PlacesHE/top_places_precalc_biodiv.geojson",driver='GeoJSON')
pd.reset_option('display.max_colwidth')

#### Don't export as shapefile because it truncates the string length

amphibians
finished MOL_ID =1925
birds
finished MOL_ID =1925
mammals
finished MOL_ID =1925
reptiles
finished MOL_ID =1925
CPU times: user 1h 24min 7s, sys: 1min 1s, total: 1h 25min 8s
Wall time: 3h 31min 18s


In [68]:
dff.head(2)

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,AFG,Afghanistan,1,60.222222,7687.504920,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1,"[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry,26.31,57.02,15.50,NaN,2.092001e+06
1,AFG,Afghanistan,2,60.222222,4706.088317,"POLYGON ((6833826.327 4162440.126, 6832851.109...",2,"[{""SliceNumber"": 9, ""per_global"": 0.2, ""per_ao...","[{""SliceNumber"": 951, ""per_global"": 4.19, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.12, ""per...","[{""SliceNumber"": 92, ""per_global"": 0.06, ""per_...",176.0,Forest,Warm Temperate Moist,1.56,55.09,5.24,NaN,6.288113e+05


In [69]:
dff.to_file("{0}/top_places_precalculated_biodiv.geojson".format(path),driver='GeoJSON')
dff.to_csv("{0}/top_places_precalculated_biodiv.csv".format(path))

In [23]:
dff.loc[dff['MOL_ID']==20,'mammals'].values[0]

'[{"SliceNumber": 575, "per_global": 1.62, "per_aoi": 87.54}, {"SliceNumber": 605, "per_global": 0.0, "per_aoi": 77.68}, {"SliceNumber": 607, "per_global": 0.01, "per_aoi": 86.61}, {"SliceNumber": 609, "per_global": 0.0, "per_aoi": 94.73}, {"SliceNumber": 615, "per_global": 0.04, "per_aoi": 67.36}, {"SliceNumber": 635, "per_global": 0.0, "per_aoi": 90.78}, {"SliceNumber": 1176, "per_global": 0.09, "per_aoi": 44.75}, {"SliceNumber": 1613, "per_global": 0.05, "per_aoi": 11.83}, {"SliceNumber": 1784, "per_global": 0.0, "per_aoi": 0.12}, {"SliceNumber": 1787, "per_global": 0.0, "per_aoi": 95.77}, {"SliceNumber": 1955, "per_global": 0.01, "per_aoi": 65.97}, {"SliceNumber": 2106, "per_global": 0.01, "per_aoi": 94.73}, {"SliceNumber": 2240, "per_global": 0.01, "per_aoi": 81.62}, {"SliceNumber": 2335, "per_global": 0.01, "per_aoi": 42.9}, {"SliceNumber": 2341, "per_global": 0.0, "per_aoi": 0.12}, {"SliceNumber": 2811, "per_global": 0.17, "per_aoi": 69.22}, {"SliceNumber": 2889, "per_global": 0

### Format contextual data 

In [91]:
bio = pd.read_csv('{0}/top_places_precalculated_biodiv.csv'.format(path)) # Here is where I bring the geojson back but it does not have the species info
bio.head(1)

,Unnamed: 0,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,0,AFG,Afghanistan,1,60.222222,7687.50492,"POLYGON ((6784878.655095577 4081461.837275224,...",1,"[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry,26.31,57.02,15.5,NaN,2.092001e+06


In [73]:
bio.loc[dff['MOL_ID']==1,'mammals'].values[0]

'[{"SliceNumber": 167, "per_global": 0.01, "per_aoi": 0.73}, {"SliceNumber": 390, "per_global": 0.0, "per_aoi": 0.09}, {"SliceNumber": 575, "per_global": 10.5, "per_aoi": 63.57}, {"SliceNumber": 605, "per_global": 0.0, "per_aoi": 10.82}, {"SliceNumber": 607, "per_global": 0.05, "per_aoi": 93.05}, {"SliceNumber": 609, "per_global": 0.01, "per_aoi": 84.46}, {"SliceNumber": 618, "per_global": 1.95, "per_aoi": 11.89}, {"SliceNumber": 1176, "per_global": 0.2, "per_aoi": 10.78}, {"SliceNumber": 1627, "per_global": 0.64, "per_aoi": 0.12}, {"SliceNumber": 1688, "per_global": 0.09, "per_aoi": 83.29}, {"SliceNumber": 1784, "per_global": 0.11, "per_aoi": 74.67}, {"SliceNumber": 1787, "per_global": 0.02, "per_aoi": 85.13}, {"SliceNumber": 1805, "per_global": 0.07, "per_aoi": 26.76}, {"SliceNumber": 2003, "per_global": 0.06, "per_aoi": 23.48}, {"SliceNumber": 2103, "per_global": 0.09, "per_aoi": 3.45}, {"SliceNumber": 2106, "per_global": 0.12, "per_aoi": 96.12}, {"SliceNumber": 2108, "per_global": 

In [30]:
### Read files (Files coming from ArcGIS project Places_HE_Future and expoprted as csv)
### Top 20 places original geometries from Scott: https://eowilson.maps.arcgis.com/home/item.html?id=4848c6b08fac4fa5bff40e9331b6d291
elu= pd.read_csv('{0}/Csvs/Places_elu.csv'.format(path))
le= pd.read_csv('{0}/Csvs/Places_encroachment.csv'.format(path))
pop= pd.read_csv('{0}/Csvs/Places_pop.csv'.format(path))
wl= pd.read_csv('{0}/Csvs/Places_wdpa_list.csv'.format(path))
wp= pd.read_csv('{0}/Csvs/Places_wdpa_percentage.csv'.format(path))

cont_lookups={'elu':'83802a7fa3d34c1fa40844fc14683966',
              'landencroach':'b0ad58e2b2404eec811b711b604b9d55'}

In [31]:
elu = elu.rename(columns ={'OBJECTID_1':'MOL_ID'})
le = le.rename(columns ={'OBJECTID_1':'MOL_ID'})
pop = pop.rename(columns ={'OBJECTID_1':'MOL_ID'})

For elu layer there are some rows that were not processed in Arcgis Pro (the geometries were not rasterized). To know which geometries are not included in elu table do this and repeat analysis in arcgis pro only for those rows.

In [32]:
l = list(range(1,1926))

In [33]:
l = list(set(dff.MOL_ID) - set(elu.MOL_ID))
len(l) # number of geometries that have no elu data, repeat analysis in ArcGIS Pro for those

95

In [34]:
# Read elu table with new rows
elu2= pd.read_csv('{0}/Csvs/Places_elu_nulls.csv'.format(path))

In [35]:
# Merge elu datasets
elu = elu.append(elu2)
elu.head(1)

,OID_,MOL_ID,COUNT,AREA,MAJORITY
0,1,1,148821.0,0.750596,184


In [36]:
l = list(set(dff.MOL_ID) - set(elu.MOL_ID))
len(l) # those geometries don't have elu data

28

In [75]:
### Column names needed: 
# 'percent_irrigated', 'percent_rainfed',
#        'percent_rangeland', 'percent_urban', 'population_sum',
#        'majority_land_cover_climate_regime', 'land_cover_majority',
#        'climate_regime_majority',

Index(['Unnamed: 0', 'GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'MOL_ID',
       'AREA_KM2', 'reptiles', 'amphibians', 'mammals', 'birds',
       'percentage_protected', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'population_sum',
       'majority_land_cover_climate_regime', 'land_cover_majority',
       'climate_regime_majority', 'country_size'],
      dtype='object')

In [37]:
dff.head(1)

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds
0,AFG,Afghanistan,1,60.222222,7687.50492,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1,"[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_..."


In [38]:
## Add contextual data: ELU
dff = dff.merge(elu[['MOL_ID','MAJORITY']], how='left', on = 'MOL_ID').rename(columns={'MAJORITY':'majority_land_cover_climate_regime'})

In [39]:
dff.head(1)

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime
0,AFG,Afghanistan,1,60.222222,7687.50492,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1,"[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0


In [40]:
# Retrieve elu lookup table to see to see the correspondences for that elu code
elu_lookup = getHTfromId(cont_lookups['elu'])
elu_lookup.head()

,elu_code,elu,lc_type,lf_type,cr_type,ObjectId
0,301,Sub Tropical Moist Forest on Plains,Forest,Plains,Sub Tropical Moist,1
1,201,Warm Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Warm Temperate Dry,2
2,151,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Cool Temperate Dry,3
3,302,Sub Tropical Moist Cropland on Tablelands,Cropland,Tablelands,Sub Tropical Moist,4
4,152,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Tablelands,Cool Temperate Dry,5


In [41]:
# Merge in dataset the required info from lookup table
dff = dff.merge(elu_lookup[['elu_code','lc_type','cr_type']], how='left', left_on = 'majority_land_cover_climate_regime', right_on = 'elu_code')\
    .drop(columns=['elu_code'])\
    .rename(columns={'lc_type':'land_cover_majority','cr_type':'climate_regime_majority'})

In [42]:
dff.head(1)

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority
0,AFG,Afghanistan,1,60.222222,7687.50492,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1,"[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry


In [43]:
# Each slicenumber with a type of land encroachment
le.head(1)

,OID_,MOL_ID,COUNT,AREA,Variable,Dimensions,SliceNumber,MIN,MAX,RANGE,MEAN,STD,SUM,MEDIAN,PCT90,percentage_land_encroachment
0,1,1,2846.0,0.197635,Variable_new,SliceNumber,1.0,0.276261,0.814483,0.538222,0.462732,0.133903,1316.934454,0.408784,0.684398,26.31


In [44]:
# See all land encroachment data for each mol_id
long =le[['SliceNumber','MOL_ID','percentage_land_encroachment']].pivot(index='MOL_ID', columns='SliceNumber', values='percentage_land_encroachment').reset_index().rename(columns=
                                                                                                                              {1.0:'percent_irrigated',
                                                                                                                              2.0:'percent_rainfed',
                                                                                                                              3.0:'percent_rangeland',
                                                                                                                                 4.0:'percent_urban'})

In [45]:
long.head(1)

SliceNumber,MOL_ID,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban
0,1,26.31,57.02,15.5,NaN


In [46]:
# How many encroachment values are higher than 100? This can be due to the way the area is calculated. Anyway, the website will show only the le that is 
# majoritary and the % will stop at 100%
long['sum']=long.iloc[:, 1:5].sum(axis=1)
long[long['sum']>100]

SliceNumber,MOL_ID,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,sum
2,3,33.07,6.93,111.57,2.66,154.23
5,7,14.04,0.93,101.45,0.14,116.56
19,22,NaN,23.95,108.24,0.74,132.93
20,23,0.67,147.48,43.64,2.01,193.80
24,28,11.90,118.87,26.94,2.91,160.62
...,...,...,...,...,...,...
1449,1825,4.79,12.90,153.44,8.54,179.67
1489,1878,1.22,50.35,106.65,10.75,168.97
1490,1879,20.71,16.55,81.57,3.66,122.49
1506,1895,NaN,NaN,139.54,NaN,139.54


In [47]:
## Add contextual data: Land Encroachment
dff = dff.merge(long[['MOL_ID','percent_irrigated', 'percent_rainfed', 'percent_rangeland',
       'percent_urban']],on ='MOL_ID',how='left')

In [74]:
dff.head(1)

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,AFG,Afghanistan,1,60.222222,7687.50492,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1,"[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry,26.31,57.02,15.5,NaN,2.092001e+06


In [50]:
# Population table, we need the SUM attribute
pop.head(1)

,OID_,MOL_ID,COUNT,AREA,SUM
0,1,1,10824.0,0.751667,2.092001e+06


In [51]:
## Add contextual data: POP
dff = dff.merge(pop[['MOL_ID','SUM']],on ='MOL_ID',how='left')
dff.head(1)

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,SUM
0,AFG,Afghanistan,1,60.222222,7687.50492,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1,"[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry,26.31,57.02,15.5,NaN,2.092001e+06


In [52]:
dff = dff.rename(columns ={'SUM':'population_sum'})
dff.head(1)

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,AFG,Afghanistan,1,60.222222,7687.50492,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1,"[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry,26.31,57.02,15.5,NaN,2.092001e+06


In [53]:
# Remove geometries that were removed from calculations in ArcGIS Pro because they gave errors when running the sample tool against the crfs
dff2 = dff.loc[(dff["MOL_ID"]!=62)] # ATL
dff2 = dff2.loc[(dff["MOL_ID"]!=69)] # ATL
dff2 = dff2.loc[(dff["MOL_ID"]!=73)] # ATL
dff2 = dff2.loc[(dff["MOL_ID"]!=194)] # BRZ
len(dff2)

1921

In [75]:
dff2.head(1)

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,AFG,Afghanistan,1,60.222222,7687.50492,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1,"[{""SliceNumber"": 9, ""per_global"": 0.29, ""per_a...","[{""SliceNumber"": 951, ""per_global"": 4.02, ""per...","[{""SliceNumber"": 167, ""per_global"": 0.01, ""per...","[{""SliceNumber"": 26, ""per_global"": 0.02, ""per_...",184.0,Cropland,Warm Temperate Dry,26.31,57.02,15.5,NaN,2.092001e+06


In [76]:
dff.loc[dff['MOL_ID']==20,'mammals'].values[0]

'[{"SliceNumber": 575, "per_global": 1.62, "per_aoi": 87.54}, {"SliceNumber": 605, "per_global": 0.0, "per_aoi": 77.68}, {"SliceNumber": 607, "per_global": 0.01, "per_aoi": 86.61}, {"SliceNumber": 609, "per_global": 0.0, "per_aoi": 94.73}, {"SliceNumber": 615, "per_global": 0.04, "per_aoi": 67.36}, {"SliceNumber": 635, "per_global": 0.0, "per_aoi": 90.78}, {"SliceNumber": 1176, "per_global": 0.09, "per_aoi": 44.75}, {"SliceNumber": 1613, "per_global": 0.05, "per_aoi": 11.83}, {"SliceNumber": 1784, "per_global": 0.0, "per_aoi": 0.12}, {"SliceNumber": 1787, "per_global": 0.0, "per_aoi": 95.77}, {"SliceNumber": 1955, "per_global": 0.01, "per_aoi": 65.97}, {"SliceNumber": 2106, "per_global": 0.01, "per_aoi": 94.73}, {"SliceNumber": 2240, "per_global": 0.01, "per_aoi": 81.62}, {"SliceNumber": 2335, "per_global": 0.01, "per_aoi": 42.9}, {"SliceNumber": 2341, "per_global": 0.0, "per_aoi": 0.12}, {"SliceNumber": 2811, "per_global": 0.17, "per_aoi": 69.22}, {"SliceNumber": 2889, "per_global": 0

In [ ]:
dff3 = dff[['GID_0', 'NAME_0', 'AREA_KM2', 'geometry', 'MOL_ID', 'reptiles', 'amphibians', 'mammals', 'birds']]

In [96]:
# dff2.to_file("{0}/top_places_precalculated.geojson".format(path),driver='GeoJSON')
dff2.to_csv("{0}/top_places_precalculated_all.csv".format(path))
# dff2.to_file("{0}/top_places_precalculated.gpkp".format(path), driver="GPKG")

In [ ]:
dff2.to_file("{0}/top_places_precalculated.gpkp".format(path), driver="GPKP")

In [ ]:
bio = pd.read_csv('{0}/top_places_precalc_biodiv.geojson'.format(path)) # Here is where I bring the geojson back but it does not have the species info
bio.head(1)

In [98]:
import sys
print(sys.getrecursionlimit())

3000


In [81]:
test.loc[dff['MOL_ID']==20,'mammals'].values[0]

'[{"SliceNumber": 575, "per_global": 1.62, "per_aoi": 87.54}, {"SliceNumber": 605, "per_global": 0.0, "per_aoi": 77.68}, {"SliceNumber": 607, "per_global": 0.01, "per_aoi": 86.61}, {"SliceNumber": 609, "per_global": 0.0, "per_aoi": 94.73}, {"SliceNumber": 615, "per_global": 0.04, "per_aoi": 67.36}, {"SliceNumber": 635, "per_global": 0.0, "per_aoi": 90.78}, {"SliceNumber": 1176, "per_global": 0.09, "per_aoi": 44.75}, {"SliceNumber": 1613, "per_global": 0.05, "per_aoi": 11.83}, {"SliceNumber": 1784, "per_global": 0.0, "per_aoi": 0.12}, {"SliceNumber": 1787, "per_global": 0.0, "per_aoi": 95.77}, {"SliceNumber": 1955, "per_global": 0.01, "per_aoi": 65.97}, {"SliceNumber": 2106, "per_global": 0.01, "per_aoi": 94.73}, {"SliceNumber": 2240, "per_global": 0.01, "per_aoi": 81.62}, {"SliceNumber": 2335, "per_global": 0.01, "per_aoi": 42.9}, {"SliceNumber": 2341, "per_global": 0.0, "per_aoi": 0.12}, {"SliceNumber": 2811, "per_global": 0.17, "per_aoi": 69.22}, {"SliceNumber": 2889, "per_global": 0

In [82]:
len(test)

1921

In [58]:
test.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich